In [1]:
!pip install -U selenium
!pip install -U pandas
!pip install -U bs4

Requirement already up-to-date: selenium in c:\users\alexi\anaconda3\lib\site-packages (3.141.0)
Requirement already up-to-date: pandas in c:\users\alexi\anaconda3\lib\site-packages (1.0.3)
Requirement already up-to-date: bs4 in c:\users\alexi\anaconda3\lib\site-packages (0.0.1)


In [2]:
# Importing Packages:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import pandas as pd
from bs4 import BeautifulSoup

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Chrome(options = options)

In [4]:
driver.get('https://www.renodepot.com/en/tools/1')
driver.refresh()

In [5]:
selectMTL = driver.find_element_by_id("department")
selectMTL.click();

In [6]:
page_source = driver.page_source

In [7]:
soup = BeautifulSoup(page_source, 'lxml')

In [8]:
print(soup.prettify())

<html class="dj_webkit dj_chrome dj_contentbox js no-flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths" lang="en">
 <head>
  <!-- BEGIN RenoCommonMetaToInclude.jspf -->
  <title>
   Tools | Réno-Dépôt
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="Tools: we've got it. Take advantage of unbeatable inventory and prices from Quebec's expert in construction &amp; renovation." name="description"/>
  <meta content="" name="keyword"/>
  <meta content="storeId_10701" name="CommerceSearch"/>
  <met

In [20]:
products = soup.find_all('div', class_ = 'product_desc')
product_df = pd.DataFrame()

for product in products:
    make = product.find('div', class_ = 'product_manufacturer').get_text().strip()
    makeTool = product.find('div', class_ = 'product_name').get_text().strip()
    normPrice = product.find('span', class_ = 'linethrough').get_text().strip()
    salePrice = product.find('span', class_ = 'integer').get_text().strip()
    print(make, makeTool, normPrice, salePrice)

METABO HPT Pneumatic Stripe Brad Nailer - 5/8"-2" $179.00 89
MAKITA Set of 2 Cordless Tools 18V $319.00 189
MAKITA Set of 2 Cordless Tools - 18 V Lithium-ion $399.00 249
BOSCH Small Angle Grinder - 4 1/2" - Set of 2 $149.00 99
BOSCH Laser Measuring Tool - 100 ft $89.99 59
BOSCH Corded Drill-Mixer - 5/8" - 9.0 A $199.00 139
MAKITA 18V LXT Cordless VSR Drill $189.00 139
MAKITA 18V Cordless Hammer Drill and Impact Driver Set - 6 pieces $399.00 299
MAKITA Set of 2 Sanders $129.00 99
BOSCH Compact Reciprocating Saw - 1'' - 12 A $189.00 149
BOSCH Woodrazor® Micrograin Carbide Planer Blades - 3 1/4'' - 2-Pack $25.89 20
BOSCH Drill/Driver Kit - 1/2" - 2 18V SlimPack Batteries $149.00 119
BOSCH Angle Grinder - Corded - 4 1/2" $69.00 55
BOSCH Variable Speed Hammer Drill - 7 A - 1/2'' $119.00 95
MAKITA 7 1/4" Teal 5200 RPM Circular Saw $119.00 95
BOSCH Compact 1/2" Drill/Driver and 1/4" Hex Impact Driver - 18 V $229.00 183
BOSCH Random Orbit Sander/Polisher - 2.5 A - 5'' $105.00 84
MAKITA Drill, 

In [ ]:
print(products)